In [1]:
from libra.prompts import LLAMAINDEX_REACT_PROMPT, SYSTEM_INIT_PROMPT

In [2]:
from libra.models import ModelBackend, ModelFactory
from libra.types import ModelLabel
from libra.config import ChatGPTConfig
from libra.functions.retrieval_functions import RETRIEVAL_FUNCS
from tqdm.autonotebook import tqdm, trange

loading embedding model...

/home/gcp_dpf/libra-core/.venv/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


finish!


In [3]:
model = ModelFactory.create(ModelLabel.GPT_4o, ChatGPTConfig(temperature=0.0).__dict__)

In [4]:
tool_dict = {tool.get_function_name(): tool.func for tool in RETRIEVAL_FUNCS}

In [5]:
tools_desc = []

for func in RETRIEVAL_FUNCS:
    tool_desc = f"""
> Tool name: {func.get_function_name()}
Tool Description: {func.get_function_description()}
Tool Args: {func.get_openai_tool_schema()['function']['parameters']}
"""
    tools_desc.append(tool_desc)
    
tool_names = [func.get_function_name() for func in RETRIEVAL_FUNCS]

In [6]:

prompt = LLAMAINDEX_REACT_PROMPT.format(
    tool_desc = '\n'.join(tools_desc),
    tool_names = ', '.join(tool_names),
    context_prompt = SYSTEM_INIT_PROMPT
)

In [7]:
print(prompt)


Your are designed to specifically help with some tasks, from answering questions to providing summaries to other types of analyses, or
even executing some function to query a database or call an API.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:

> Tool name: mb_information_retrieval
Tool Description: Retrieves MB Bank information based on a Vietnamese query.
Use when user ask anything about MB (MB Bank) such as: MB Bank's history,
MB Bank's products and services, MB Bank's achievement, MB Bank's officer salary, any problem relating to MB.  
Furthermore, if user want to receive advice about career path, use this function to add more context to your answer.
Tool Args: {'properties': {'query': {'type': 'string', 'description': 'A Vie

In [16]:
input_text = "tôi vừa bị sa thải, lại còn dính nợ xấu, giờ phải trốn khắp nơi, giờ phải làm gì để kiếm tiền trả nợ?"

response = model.run(messages=[
    {"role": "system", "content": prompt},
    {"role": "user", "content": input_text}
])

text_response = ""
for chunk in response:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")
        text_response += chunk.choices[0].delta.content

Thought: The user's question is about personal financial difficulties and seeking advice on how to earn money to pay off debts. This is not directly related to MB Bank's career paths, job opportunities, or specific information about MB Bank.

Answer: Mình rất tiếc khi nghe về tình huống khó khăn của bạn. Tuy nhiên, mình không thể cung cấp lời khuyên tài chính cá nhân hoặc các giải pháp cụ thể cho vấn đề nợ xấu. Bạn có thể tìm kiếm sự tư vấn từ các chuyên gia tài chính hoặc các tổ chức hỗ trợ tài chính để nhận được sự giúp đỡ phù hợp. Nếu bạn quan tâm đến các cơ hội việc làm tại MB Bank, mình có thể giúp bạn tìm kiếm thông tin về các vị trí tuyển dụng hiện có. Bạn có muốn mình giúp không?

In [13]:
import json

lines = text_response.split("\n")

for line in lines:
    if line.startswith("Action: "):
        action = line.replace("Action: ", "")
    if line.startswith("Action Input: "):
        action_input = line.replace("Action Input: ", "")
        try:
            action_input = json.loads(action_input)
        except json.JSONDecodeError:
            raise ValueError("Invalid JSON in action input")
            

In [14]:
action, action_input

('job_retrieval', {'workplace': 'Hà Nội', 'industry': 'Dịch vụ khách hàng'})

In [18]:
results = tool_dict[action](**action_input) # type: ignore

print(results)

profile mong muốn:
{'workplace': 'Hà Nội', 'industry': 'Dịch vụ khách hàng', 'welfare': '', 'min_salary': 10000000, 'max_salary': 35000000}
Các jobs tìm được:
{'_id': {'$oid': '66c015512d62a130f8080359'}, 'url': 'https://tuyendung.mbbank.com.vn/viec-lam/chuyen-vien-khach-hang-ca-nhan-ha-noi.35aa8968.html', 'title': 'Chuyên viên Khách hàng Cá nhân - Hà Nội', 'workplace': 'Hà Nội', 'job_rank': 'Nhân viên', 'job_form': 'Nhân viên chính thức', 'experience': '1 Năm', 'salary': '12,000,000 - 35,000,000  VNĐ', 'industry': 'Ngân hàng, Tài chính / Đầu tư', 'deadline_applications': '16/06/2024', 'welfare': ['Máy tính xách tay', 'Bảo hiểm', 'Du Lịch', 'Phụ cấp', 'Du lịch nước ngoài', 'Đồng phục', 'Thưởng', 'Chăm sóc sức khỏe', 'Đào tạo', 'Tăng lương', 'Công tác phí', 'Phụ cấp thâm niên', 'Chế độ nghỉ phép', 'Chế độ tập thể dục'], 'job_detail': ['Lập kế hoạch và triển khai các kế hoạch kinh doanh của cá nhân, tập trung vào các sản phẩm tín dụng, cho vay Sản xuất kinh doanh, Bất động sản, Dự án, ti

In [19]:
messages=[
    {"role": "system", "content": prompt},
    {"role": "user", "content": input_text},
    {"role": "assistant", "content": text_response},
    {"role": "user", "content": f"Observation:  tài liệu được tìm thấy: {results}"}
]

response = model.run(messages) # type: ignore

text_response = ""
for chunk in response:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end='')
        text_response += chunk.choices[0].delta.content

Thought: I have found some job opportunities in the "Dịch vụ khách hàng" industry at MB Bank in Hà Nội. I'll provide the details to the user.

Answer: Tớ đã tìm thấy một số vị trí công việc thú vị trong ngành **Dịch vụ khách hàng** tại MB Bank ở Hà Nội. Dưới đây là thông tin chi tiết:

1. **[Chuyên viên Khách hàng Cá nhân - Hà Nội](https://tuyendung.mbbank.com.vn/viec-lam/chuyen-vien-khach-hang-ca-nhan-ha-noi.35aa8968.html)**
   - **Mức lương**: 12,000,000 - 35,000,000 VNĐ
   - **Kinh nghiệm**: 1 Năm
   - **Phúc lợi**: Máy tính xách tay, Bảo hiểm, Du Lịch, Phụ cấp, Du lịch nước ngoài, Đồng phục, Thưởng, Chăm sóc sức khỏe, Đào tạo, Tăng lương, Công tác phí, Phụ cấp thâm niên, Chế độ nghỉ phép, Chế độ tập thể dục
   - **Yêu cầu**: Tốt nghiệp Đại học chuyên ngành Tài chính, Ngân hàng, Kinh tế; Kỹ năng bán hàng, kỹ năng giao tiếp tốt; Thành thạo tin học văn phòng cơ bản (word, excel, outlook…)

2. **[Chuyên viên Khách hàng Doanh nghiệp (SME) - Khu vực Hà Nội](https://tuyendung.mbbank.com.v